In [10]:
import ipywidgets as widgets
import sys
from pathlib import Path
import os
import importlib


module_path='preprocessing/day_intervals_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)

module_path='utils'
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path='preprocessing/hosp_module_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path='model'
if module_path not in sys.path:
    sys.path.append(module_path)

root_dir = "/data/corpora_alpha/MIMIC/MIMIC_IV_2.2/files"
import day_intervals_cohort
from day_intervals_cohort import *

import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

import data_generation_icu

import data_generation
import evaluation

import feature_selection_hosp
from feature_selection_hosp import *

# import train
# from train import *


import ml_models
from ml_models import *

import dl_train
from dl_train import *

import tokenization
from tokenization import *


import behrt_train
from behrt_train import *

import feature_selection_icu
from feature_selection_icu import *
import fairness
import callibrate_output

In [11]:
importlib.reload(day_intervals_cohort)
import day_intervals_cohort
from day_intervals_cohort import *

importlib.reload(day_intervals_cohort_v2)
import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

importlib.reload(data_generation_icu)
import data_generation_icu
importlib.reload(data_generation)
import data_generation

importlib.reload(feature_selection_hosp)
import feature_selection_hosp
from feature_selection_hosp import *

importlib.reload(feature_selection_icu)
import feature_selection_icu
from feature_selection_icu import *

importlib.reload(tokenization)
import tokenization
from tokenization import *

importlib.reload(ml_models)
import ml_models
from ml_models import *

importlib.reload(dl_train)
import dl_train
from dl_train import *

importlib.reload(behrt_train)
import behrt_train
from behrt_train import *

importlib.reload(fairness)
import fairness

importlib.reload(callibrate_output)
import callibrate_output

importlib.reload(evaluation)
import evaluation

In [12]:
def eda_basic(dataframes, dataframes_names = None, log_path=None):
    '''
    Perform basic exploratory data analysis on a list of pandas DataFrames.
    Args:
    dataframes (list): A list of pandas DataFrames.
    dataframes_names (list, optional): A list of names for the DataFrames. If provided, the names will be used to distinguish between the DataFrames in the results.
    log_path (str, optional): The path to save the log file. If provided, the results will be logged to the file.
    '''
    if(len(dataframes) == 0):
        print("No dataframes provided.")
        return
    
    if dataframes_names:
        if len(dataframes) != len(dataframes_names):
            print("The number of dataframes and the number of names provided do not match.")
            return

    if log_path:
        log_file_path = os.path.join(log_path, 'eda_basic_log.txt')
        os.makedirs(os.path.dirname(log_file_path), exist_ok=True)
        log_file = open(log_file_path, 'a')
        log_file.write(f"{'='*40}\n")
        log_file.write(f"{pd.Timestamp.now()}\n")
        log_file.write(f"{'='*40}\n")
        
    for df_index in range(len(dataframes)):
        if dataframes_names:
            print(f"{'*'*40}\n{dataframes_names[df_index]}\n{'*'*40}")
            if log_path:
                log_file.write(f"{'*'*40}\n{dataframes_names[df_index]}\n{'*'*40}\n")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
            results = [
                ("First few rows of the dataframe:\n", dataframes[df_index].head()),
                ("Columns in the dataframe:\n", dataframes[df_index].columns),
                ("Number of rows in the dataframe:\n", dataframes[df_index].shape[0]),
                ("Missing values in the dataframe:\n", dataframes[df_index].isnull().sum()),
                ("Number of duplicate rows in the dataframe:\n", dataframes[df_index].duplicated().sum()),
                ("Data types of columns:\n", dataframes[df_index].dtypes),
                ("Number of unique values in each column:\n", dataframes[df_index].nunique()),
            ]
            for label, result in results:
                print(label)
                print(result)
                print()
                if log_path:
                    log_file.write(f"{label}{result}\n\n")
            if log_path:
                log_file.write(f"{'X'*40}\n")
    if log_path:
        log_file.close()

In [13]:
imputation_cohort_var = day_intervals_cohort_v2.extract_imputation_data("Non-ICU", root_dir)
imputation_cohort_dataset_non_ICU = pd.read_csv(f"./data/cohort/{imputation_cohort_var}.csv.gz")

EXTRACTING FOR: | NON-ICU | IMPUTATION |


In [14]:
imputation_cohort_var = day_intervals_cohort_v2.extract_imputation_data("ICU", root_dir)
imputation_cohort_dataset_ICU = pd.read_csv(f"./data/cohort/{imputation_cohort_var}.csv.gz")

EXTRACTING FOR: | ICU | IMPUTATION |


In [17]:
eda_basic([imputation_cohort_dataset_non_ICU, imputation_cohort_dataset_ICU], ['Non-ICU', 'ICU'], './data/summary/')

****************************************
Non-ICU
****************************************
First few rows of the dataframe:

   subject_id   hadm_id            admittime            dischtime deathtime  \
0    10000032  22595853  2180-05-06 22:23:00  2180-05-07 17:15:00       NaN   
1    10000032  22841357  2180-06-26 18:27:00  2180-06-27 18:49:00       NaN   
2    10000032  25742920  2180-08-05 23:44:00  2180-08-07 17:50:00       NaN   
3    10000032  29079034  2180-07-23 12:35:00  2180-07-25 17:55:00       NaN   
4    10000068  25022803  2160-03-03 23:16:00  2160-03-04 06:26:00       NaN   

   admission_type admit_provider_id      admission_location  \
0          URGENT            P874LG  TRANSFER FROM HOSPITAL   
1        EW EMER.            P09Q6Y          EMERGENCY ROOM   
2        EW EMER.            P60CC5          EMERGENCY ROOM   
3        EW EMER.            P30KEH          EMERGENCY ROOM   
4  EU OBSERVATION            P51VDL          EMERGENCY ROOM   

  discharge_location i

#### So till here we are able to create the cohort with patient specific details -- Now we can move onto the features per visit

#### Decide which columns to drop (i.e they won't be a part of the dataset itself -- makes sense to choose those with very high missing values) 

#### For the reaminder of the missing values in columns -- we will drop the column as we want to start with a base of all available cohort